In [5]:
import pandas as pd
import numpy as np
from collections import Counter




In [6]:
# Simulate a skewed "transactions" dataset
np.random.seed(42)
user_ids = np.random.choice(
    [1, 2, 3, 4, 5, 999],  # user 999 will be our "skewed" key
    size=10000,
    p=[0.05, 0.05, 0.05, 0.05, 0.05, 0.75]  # 75% of rows are user 999
)


In [7]:
transactions = pd.DataFrame({
    'user_id': user_ids,
    'amount': np.random.randint(10, 500, size=10000)
})

In [9]:
df_counts = transactions.groupby('user_id').size().reset_index(name='n_rows').sort_values('n_rows', ascending=False)
df_counts

,user_id,n_rows
5,999,7453
1,2,538
3,4,509
4,5,504
2,3,503
0,1,493


In [ ]:


# Small users dataset
users = pd.DataFrame({
    'user_id': [1, 2, 3, 4, 5, 999],
    'country': ['US', 'CA', 'FR', 'DE', 'UK', 'IN']
})

print("Transaction distribution by user:")
print(Counter(transactions['user_id']))